In [3]:
%%capture
!wget https://github.com/skoltech-nlp/detox/releases/download/emnlp2021/filtered_paranmt.zip
!unzip filtered_paranmt.zip

In [1]:
%%capture
!pip install transformers[torch] datasets evaluate sacrebleu
!pip install accelerate -U
!pip install wandb

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, DataCollatorForSeq2Seq
import evaluate

import numpy as np
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import os
from transformers import pipeline

In [4]:
df = pd.read_csv('filtered.tsv', sep='\t').drop(columns=['Unnamed: 0'])
df

,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,"If Alkar is flooding her with psychic waste, t...","if Alkar floods her with her mental waste, it ...",0.785171,0.010309,0.014195,0.981983
1,Now you're getting nasty.,you're becoming disgusting.,0.749687,0.071429,0.065473,0.999039
2,"Well, we could spare your life, for one.","well, we can spare your life.",0.919051,0.268293,0.213313,0.985068
3,"Ah! Monkey, you've got to snap out of it.","monkey, you have to wake up.",0.664333,0.309524,0.053362,0.994215
4,I've got orders to put her down.,I have orders to kill her.,0.726639,0.181818,0.009402,0.999348
...,...,...,...,...,...,...
577772,You didn't know that Estelle had stolen some f...,you didn't know that Estelle stole your fish f...,0.870322,0.030769,0.000121,0.949143
577773,It'il suck the life out of you!,you'd be sucked out of your life!,0.722897,0.058824,0.996124,0.215794
577774,"I can't fuckin' take that, bruv.",I really can't take this.,0.617511,0.212121,0.984538,0.000049
577775,They called me a fucking hero. The truth is I ...,"they said I was a hero, but I didn't care.",0.679613,0.358209,0.991945,0.000124


In [5]:
(df.ref_tox < df.trn_tox).mean()

0.4476381025897535

In [6]:
cp_reference = df.reference.copy()
cp_tox_reference = df.ref_tox.copy()
mask = df.ref_tox < df.trn_tox
df.loc[mask, 'reference'] = df.loc[mask, 'translation']
df.loc[mask, 'translation'] = cp_reference

df.loc[mask, 'ref_tox'] = df.loc[mask, 'trn_tox']
df.loc[mask, 'trn_tox'] = cp_tox_reference

In [7]:
(df.ref_tox < df.trn_tox).mean()

0.0

In [8]:
df

,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,"if Alkar floods her with her mental waste, it ...","If Alkar is flooding her with psychic waste, t...",0.785171,0.010309,0.981983,0.014195
1,you're becoming disgusting.,Now you're getting nasty.,0.749687,0.071429,0.999039,0.065473
2,"well, we can spare your life.","Well, we could spare your life, for one.",0.919051,0.268293,0.985068,0.213313
3,"monkey, you have to wake up.","Ah! Monkey, you've got to snap out of it.",0.664333,0.309524,0.994215,0.053362
4,I have orders to kill her.,I've got orders to put her down.,0.726639,0.181818,0.999348,0.009402
...,...,...,...,...,...,...
577772,you didn't know that Estelle stole your fish f...,You didn't know that Estelle had stolen some f...,0.870322,0.030769,0.949143,0.000121
577773,It'il suck the life out of you!,you'd be sucked out of your life!,0.722897,0.058824,0.996124,0.215794
577774,"I can't fuckin' take that, bruv.",I really can't take this.,0.617511,0.212121,0.984538,0.000049
577775,They called me a fucking hero. The truth is I ...,"they said I was a hero, but I didn't care.",0.679613,0.358209,0.991945,0.000124


Split Dataset into train and test subsets

In [9]:
hf_df = Dataset.from_pandas(df)
hf_df = hf_df.train_test_split(test_size=0.2)

In [10]:
hf_df

DatasetDict({
    train: Dataset({
        features: ['reference', 'translation', 'similarity', 'lenght_diff', 'ref_tox', 'trn_tox'],
        num_rows: 462221
    })
    test: Dataset({
        features: ['reference', 'translation', 'similarity', 'lenght_diff', 'ref_tox', 'trn_tox'],
        num_rows: 115556
    })
})

Translate the data into an appropriate format for T5 fine-tuning.

In [11]:

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [12]:
prefix = "Perform Text-Detoxification: "


def preprocess_function(examples):
    inputs = [prefix + example for example in examples['reference']]
    targets = [example for example in examples['translation']]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [13]:
hf_df = hf_df.map(preprocess_function, batched=True)

Map:   0%|          | 0/462221 [00:00<?, ? examples/s]

Map:   0%|          | 0/115556 [00:00<?, ? examples/s]

Create Collator object for dynamic text padding

In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

Metric Evaluation Code

In [15]:
metric = evaluate.load("sacrebleu")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

Fine-Tune the model

In [16]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [17]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [18]:
os.environ["WANDB_PROJECT"] = "text_detoxification"
os.environ["WANDB_GROUP"] = "T5_seq2seq"
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

training_args = Seq2SeqTrainingArguments(
    output_dir="output",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    report_to="wandb"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=hf_df["train"],
    eval_dataset=hf_df["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: Currently logged in as: ummagumm-a. Use `wandb login --relogin` to force relogin


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.983500,1.835113,22.168600,13.310600
2,1.938400,1.814326,22.322700,13.290600


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=28890, training_loss=2.0053932324624384, metrics={'train_runtime': 6284.9908, 'train_samples_per_second': 147.087, 'train_steps_per_second': 4.597, 'total_flos': 1.359414423109632e+16, 'train_loss': 2.0053932324624384, 'epoch': 2.0})

In [19]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

'https://huggingface.co/ummagumm-a/output/tree/main/'

In [15]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [18]:

detoxifier = pipeline("translation", model="ummagumm-a/output", revision='44a0213')

Examples of toxic texts where the model shines in its cleaning abilities.

In [21]:
for i in [5, 10, 12, 13, 14, 15]:
    print('-'*100)
    print("REFERENCE:", hf_df['test'][i]['reference'])
    print("TRANSLATION:", hf_df['test'][i]['translation'])
    print("MODEL TRANSLATION:", detoxifier(hf_df['test'][i]['reference']))

----------------------------------------------------------------------------------------------------
REFERENCE: I thought you said you knew this fucking guy.
TRANSLATION: didn't you say you knew him?
MODEL TRANSLATION: [{'translation_text': 'I thought you said you knew this guy.'}]
----------------------------------------------------------------------------------------------------
REFERENCE: one of my guys is squirming 10 to 15 years old for some kind of electronic voodoo shit from the Feds.
TRANSLATION: My boy tiny's doing 10 to 15 because of some electronic voodoo the Feds pulled.
MODEL TRANSLATION: [{'translation_text': 'one of my guys is squirming from 10 to 15 years old for some kind of electronic voodoo nonsense from the Feds.'}]
----------------------------------------------------------------------------------------------------
REFERENCE: You know how I know you fucked him?
TRANSLATION: you know how I know you drove him?
MODEL TRANSLATION: [{'translation_text': 'You know how I k